In [1]:
import datajoint as dj
import os
from pathlib import Path

# set dirs
base_dir = Path('/Users/rly/Documents/NWB/spyglass-workspace') # change this to your desired directory
# base_dir = Path('/hdd/dj') # change this to your desired directory
raw_dir = base_dir / 'raw'
analysis_dir = base_dir / 'analysis'
recording_dir = base_dir / 'recording'
sorting_dir = base_dir / 'sorting'
waveforms_dir = base_dir / 'waveforms'
tmp_dir = base_dir / 'tmp'
kachery_cloud_dir = base_dir / '.kachery_cloud'

os.makedirs(raw_dir, exist_ok=True)
os.makedirs(analysis_dir, exist_ok=True)
os.makedirs(recording_dir, exist_ok=True)
os.makedirs(sorting_dir, exist_ok=True)
os.makedirs(waveforms_dir, exist_ok=True)
os.makedirs(tmp_dir, exist_ok=True)
os.makedirs(kachery_cloud_dir, exist_ok=True)

# set dj config
dj.config['database.host'] = 'localhost'
dj.config['database.user'] = 'root'
dj.config['database.password'] = 'tutorial'
dj.config['database.port'] = 3306
dj.config['stores'] = {
  'raw': {
    'protocol': 'file',
    'location': str(raw_dir),
    'stage': str(raw_dir)
  },
  'analysis': {
    'protocol': 'file',
    'location': str(analysis_dir),
    'stage': str(analysis_dir)
  }
}
dj.config["enable_python_native_blobs"] = True

# set env vars
os.environ['SPYGLASS_BASE_DIR'] = str(base_dir)
os.environ['SPYGLASS_RECORDING_DIR'] = str(recording_dir)
os.environ['SPYGLASS_SORTING_DIR'] = str(sorting_dir)
os.environ['SPYGLASS_WAVEFORMS_DIR'] = str(waveforms_dir)
os.environ['SPYGLASS_TEMP_DIR'] = str(tmp_dir)
os.environ['KACHERY_CLOUD_DIR'] = str(kachery_cloud_dir)
os.environ['DJ_SUPPORT_FILEPATH_MANAGEMENT'] = 'TRUE'

import spyglass as sg
import spyglass.common as sgc
import spyglass.data_import as sgdi
import spyglass.spikesorting as sgs
import warnings
warnings.simplefilter("always")

[2023-02-06 17:59:47,562][INFO]: Connecting root@localhost:3306
[2023-02-06 17:59:47,721][INFO]: Connected root@localhost:3306
/Users/rly/mambaforge/envs/spyglass/lib/python3.9/site-packages/position_tools/core.py:3: DeprecationWarning: Please use `gaussian_filter1d` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter1d
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [2]:
sgc.DataAcquisitionDeviceSystem().delete(safemode=False)
sgc.DataAcquisitionDeviceAmplifier().delete(safemode=False)
sgc.DataAcquisitionDevice().delete(safemode=False)
sgc.Institution().delete(safemode=False, force_parts=True)
sgc.Lab().delete(safemode=False)
sgc.LabTeam().delete(safemode=False)
sgc.LabMember().delete(safemode=False)
sgc.Subject().delete(safemode=False)
sgc.Probe().delete(safemode=False)
sgc.ProbeType().delete(safemode=False)
sgc.Session().delete(safemode=False)
sgc.Nwbfile().delete(safemode=False)
sgc.Nwbfile().cleanup(delete_files=True)
sgs.SortGroup().delete(safemode=False)
sgs.SortInterval().delete(safemode=False)
sgs.SpikeSorterParameters().delete(safemode=False)
sgs.SpikeSortingPreprocessingParameters().delete(safemode=False)
sgs.SpikeSortingRecordingSelection().delete(safemode=False)
sgs.Curation().delete(safemode=False)
sgc.BrainRegion().delete(safemode=False)

[2023-02-06 17:59:49,667][INFO]: Deleting 0 rows from `common_device`.`data_acquisition_device_system`
[2023-02-06 17:59:49,684][INFO]: Deleting 0 rows from `common_device`.`data_acquisition_device_amplifier`
[2023-02-06 17:59:49,704][INFO]: Deleting 0 rows from `common_device`.`data_acquisition_device`
[2023-02-06 17:59:49,777][INFO]: Deleting 384 rows from `common_ephys`.`_electrode`
[2023-02-06 17:59:49,789][INFO]: Deleting 1 rows from `common_ephys`.`_electrode_group`
[2023-02-06 17:59:49,810][INFO]: Deleting 1 rows from `common_ephys`.`_raw`
[2023-02-06 17:59:49,830][INFO]: Deleting 1 rows from `common_interval`.`interval_list`
[2023-02-06 17:59:49,853][INFO]: Deleting 1 rows from `common_session`.`_session__experimenter`
[2023-02-06 17:59:49,872][INFO]: Deleting 1 rows from `common_session`.`_session`
[2023-02-06 17:59:49,893][INFO]: Deleting 1 rows from `common_lab`.`institution`
[2023-02-06 17:59:49,919][INFO]: Deleting 1 rows from `common_lab`.`lab`
[2023-02-06 17:59:49,949][I

1

In [ ]:
sgc.DataAcquisitionDeviceSystem.insert1(
    {"data_acquisition_device_system": "Neuropixels Acquisition System"}, 
    skip_duplicates=True
)

In [ ]:
# sgc.ProbeType.insert1({
#   "probe_type": "128c-4s6mm6cm-15um-26um-sl",
#   "probe_description": "A Livermore flexible probe with 128 channels, 4 shanks, 6 mm shank length, 6 cm ribbon length. 15 um contact diameter, 26 um center-to-center distance (pitch), single-line configuration.",
#   "manufacturer": "Lawrence Livermore National Lab",
#   "num_shanks": 4,
# }, skip_duplicates=True)
# sgc.ProbeType()

In [3]:
# Define the name of the file that you copied and renamed; make sure it's something unique. 
# nwb_file_name = 'sub-CuffLinks_ses-CuffLinks-LT03-g3_behavior+ecephys.nwb'
nwb_file_name = 'sub-AppleBottom_ses-AppleBottom-DY20-g3_behavior+ecephys.nwb'

filename, file_extension = os.path.splitext(nwb_file_name)
# This is a copy of the original nwb file, except it doesn't contain the raw data (for storage reasons)
nwb_file_name2 = filename + '_' + file_extension

In [ ]:
sgc.ProbeType & {"probe_type": "Neuropixels 1.0"}

In [ ]:
sgc.Probe.create_from_nwbfile(
    nwb_file_name=nwb_file_name,
    nwb_device_name="Device",
    probe_id="Neuropixels 1.0 Giocomo Lab Configuration",
    probe_type="Neuropixels 1.0",
    contact_side_numbering=True
)

In [ ]:
sgc.Probe.Shank()

In [ ]:
sgc.Probe.Electrode()

In [4]:
sgdi.insert_sessions(nwb_file_name)

Creating a copy of NWB file sub-AppleBottom_ses-AppleBottom-DY20-g3_behavior+ecephys.nwb with link to raw ephys data: sub-AppleBottom_ses-AppleBottom-DY20-g3_behavior+ecephys_.nwb


/Users/rly/mambaforge/envs/spyglass/lib/python3.9/site-packages/datajoint/hash.py:39: ResourceWarning: unclosed file <_io.BufferedReader name='/Users/rly/Documents/NWB/spyglass-workspace/raw/sub-AppleBottom_ses-AppleBottom-DY20-g3_behavior+ecephys_.nwb'>
  return uuid_from_stream(Path(filepath).open("rb"), init_string=init_string)
/Users/rly/mambaforge/envs/spyglass/lib/python3.9/site-packages/datajoint/external.py:276: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if check_hash:


Populate Session...
Institution...
Lab...
LabMember...
Please add the Google user ID for Emily A. Aery Jones in the LabMember.LabMemberInfo table if you want to give them permission to manually curate sortings by this team.
Subject...
Populate DataAcquisitionDevice...
Inserted data acquisition device(s): {'Neuropixels Recording Device'}

Populate CameraDevice...
No conforming camera device metadata found.

Populate Probe...
No conforming probe metadata found.

Skipping Apparatus for now...
IntervalList...
No epochs found in NWB file.
DataAcquisitionDevice with name Neuropixels Recording Device does not exist. Cannot link Session with DataAcquisitionDevice in Session.DataAcquisitionDevice.
Populate ElectrodeGroup...
Populate Electrode...
Populate Raw...
Importing raw data: Sampling rate:	30000.0 Hz
Number of valid intervals:	1
Populate SampleCount...
Unable to import SampleCount: no data interface named "sample_count" found in sub-AppleBottom_ses-AppleBottom-DY20-g3_behavior+ecephys_.nw

In [ ]:
%debug

In [ ]:
sgc.Probe()

In [ ]:
sgc.Probe.Shank()

In [ ]:
sgc.Probe.Electrode()

In [ ]:
dj.ERD(sgc.Session)+1